In [1]:
pip install tensorflow opencv-python

  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl (332.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory

# ✅ Path should directly contain 'REAL' and 'FAKE' folders
dataset_path = r"C:\archive (1)\real_vs_fake\real-vs-fake"

IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 5

train_dataset = image_dataset_from_directory(
    dataset_path,
    labels="inferred",
    label_mode="binary",
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Normalize images
train_dataset = train_dataset.map(lambda x, y: (x / 255.0, y))

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile and train
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_dataset, epochs=EPOCHS)

# Save the model
model.save("liveness_model.h5")
print("[✅] Training done. Model saved as 'liveness_model.h5'")


Found 32717 files belonging to 2 classes.


C:\Users\Suraj Garole\Conda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1023/1023 ━━━━━━━━━━━━━━━━━━━━ 2074s 2s/step - accuracy: 0.6102 - loss: 0.6641
Epoch 2/5
1023/1023 ━━━━━━━━━━━━━━━━━━━━ 659s 643ms/step - accuracy: 0.6113 - loss: 0.6485 
Epoch 3/5
1023/1023 ━━━━━━━━━━━━━━━━━━━━ 676s 637ms/step - accuracy: 0.6114 - loss: 0.6369 
Epoch 4/5
1023/1023 ━━━━━━━━━━━━━━━━━━━━ 603s 589ms/step - accuracy: 0.6112 - loss: 0.6257
Epoch 5/5
1023/1023 ━━━━━━━━━━━━━━━━━━━━ 549s 537ms/step - accuracy: 0.6286 - loss: 0.6121


[✅] Training done. Model saved as 'liveness_model.h5'


In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# ✅ Load your pre-trained model
model_path = "liveness_model.h5"
try:
    model = load_model(model_path)
except Exception as e:
    print(f"[ERROR] Could not load model from '{model_path}'.\nReason: {e}")
    raise SystemExit

# ✅ Set image size as per training config
IMG_SIZE = 64

# ✅ Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# ✅ Open webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("[ERROR] Could not access webcam.")
    raise SystemExit

print("[INFO] Webcam started. Show a face to detect real or fake. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("[ERROR] Failed to read frame from webcam.")
        break

    # 🔹 Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 🔹 Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)  # Convert BGR → RGB
        face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
        face = img_to_array(face)
        face = np.expand_dims(face, axis=0)
        face = face / 255.0  # Normalize

        # 🔹 Model prediction
        prediction = model.predict(face, verbose=0)[0][0]
        label = "Real" if prediction >= 0.5 else "Fake"
        color = (0, 255, 0) if label == "Real" else (0, 0, 255)
        confidence = prediction * 100 if label == "Real" else (1 - prediction) * 100
        text = f"{label} ({confidence:.2f}%)"

        # 🔹 Draw rectangle & label
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, text, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # 🔹 Show output
    cv2.imshow("Liveness Detection", frame)

    # 🔹 Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ✅ Clean up
cap.release()
cv2.destroyAllWindows()


[INFO] Webcam started. Show a face to detect real or fake. Press 'q' to quit.


In [9]:
!pip install mediapipe


ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)
ERROR: No matching distribution found for mediapipe


In [10]:
import cv2
import numpy as np
import mediapipe as mp
import time
from collections import deque
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from scipy.spatial import distance as dist

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

EAR_THRESHOLD = 0.21
CONSEC_FRAMES = 3
MIN_BLINK_RATE = 5  # per minute
IMG_SIZE = 64
PREDICTION_HISTORY_SIZE = 10

model = load_model("liveness_model.h5")

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, max_num_faces=1)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

blink_counter = 0
total_blinks = 0
start_time = time.time()

prediction_queue = deque(maxlen=PREDICTION_HISTORY_SIZE)
last_label = "Analyzing..."
last_confidence = 0.0

cap = cv2.VideoCapture(0)
print("[INFO] Press 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye = [(int(face_landmarks.landmark[i].x * w), int(face_landmarks.landmark[i].y * h)) for i in LEFT_EYE]
            right_eye = [(int(face_landmarks.landmark[i].x * w), int(face_landmarks.landmark[i].y * h)) for i in RIGHT_EYE]

            for pt in left_eye + right_eye:
                cv2.circle(frame, pt, 2, (0, 255, 0), -1)

            ear = (eye_aspect_ratio(left_eye) + eye_aspect_ratio(right_eye)) / 2.0
            if ear < EAR_THRESHOLD:
                blink_counter += 1
            else:
                if blink_counter >= CONSEC_FRAMES:
                    total_blinks += 1
                blink_counter = 0

            x_vals = [int(p.x * w) for p in face_landmarks.landmark]
            y_vals = [int(p.y * h) for p in face_landmarks.landmark]
            x_min, x_max = min(x_vals), max(x_vals)
            y_min, y_max = min(y_vals), max(y_vals)

            if x_max - x_min > 0 and y_max - y_min > 0:
                face_crop = frame[y_min:y_max, x_min:x_max]
                try:
                    face_crop = cv2.resize(face_crop, (IMG_SIZE, IMG_SIZE))
                    face_crop = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
                    face_crop = img_to_array(face_crop)
                    face_crop = np.expand_dims(face_crop, axis=0) / 255.0
                    prediction = model.predict(face_crop, verbose=0)[0][0]
                    prediction_queue.append(prediction)

                    if len(prediction_queue) == PREDICTION_HISTORY_SIZE:
                        avg_pred = np.mean(prediction_queue)
                        blink_rate = total_blinks / ((time.time() - start_time) / 60)
                        if avg_pred >= 0.5 and blink_rate >= MIN_BLINK_RATE:
                            last_label = "Real"
                            last_confidence = avg_pred * 100
                        else:
                            last_label = "Fake"
                            last_confidence = (1 - avg_pred) * 100

                except Exception as e:
                    print(f"[WARN] Prediction error: {e}")

            color = (0, 255, 0) if last_label == "Real" else (0, 0, 255)
            cv2.putText(frame, f"{last_label} ({last_confidence:.2f}%)", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            cv2.putText(frame, f"Blinks: {total_blinks}", (10, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            blink_rate = total_blinks / ((time.time() - start_time) / 60)
            cv2.putText(frame, f"Blink Rate: {blink_rate:.2f}/min", (10, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    cv2.imshow("Stable Liveness Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'mediapipe'

In [11]:
import sys
print(sys.executable)


C:\Users\Suraj Garole\Conda\python.exe


In [4]:
import cv2
import mediapipe as mp

# Initialize Mediapipe Hands and Drawing modules
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Open webcam
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
) as hands:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally for a mirror effect
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to detect hands
        results = hands.process(frame_rgb)

        # Draw hand landmarks
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )

        # Display the output
        cv2.imshow("Hand Tracking", frame)

        # Press ESC to exit
        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'mediapipe'

In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# ---------------- Load Pre-trained Model ----------------
model_path = "liveness_model.h5"  # Make sure this file exists in the same folder
try:
    model = load_model(model_path)
except Exception as e:
    print(f"[ERROR] Could not load model from '{model_path}'.\nReason: {e}")
    exit()

# ---------------- Config ----------------
IMG_SIZE = 64  # Image size used during training
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# ---------------- Initialize Webcam ----------------
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("[ERROR] Could not access webcam.")
    exit()

print("[INFO] Webcam started. Show a face to detect real or fake. Press 'q' to quit.")

# ---------------- Main Loop ----------------
while True:
    ret, frame = cap.read()
    if not ret:
        print("[ERROR] Failed to read frame from webcam.")
        break

    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract face region
        face = frame[y:y+h, x:x+w]
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face_resized = cv2.resize(face_rgb, (IMG_SIZE, IMG_SIZE))
        face_array = img_to_array(face_resized)
        face_array = np.expand_dims(face_array, axis=0)
        face_array = face_array / 255.0  # Normalize

        # Predict
        prediction = model.predict(face_array, verbose=0)[0][0]
        label = "Real" if prediction >= 0.5 else "Fake"
        color = (0, 255, 0) if label == "Real" else (0, 0, 255)
        confidence = prediction * 100 if label == "Real" else (1 - prediction) * 100
        text = f"{label} ({confidence:.2f}%)"

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, text, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Show frame
    cv2.imshow("Liveness Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# ---------------- Cleanup ----------------
cap.release()
cv2.destroyAllWindows()
print("[INFO] Webcam closed.")


[INFO] Webcam started. Show a face to detect real or fake. Press 'q' to quit.
[INFO] Webcam closed.


In [16]:
import cv2
import numpy as np
import time
from collections import deque
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from scipy.spatial import distance as dist

# EAR calculation
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

EAR_THRESHOLD = 0.21
CONSEC_FRAMES = 3
MIN_BLINK_RATE = 5
IMG_SIZE = 64
PREDICTION_HISTORY_SIZE = 10

model = load_model("liveness_model.h5")

# Haar cascades (already in OpenCV)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

blink_counter = 0
total_blinks = 0
start_time = time.time()

prediction_queue = deque(maxlen=PREDICTION_HISTORY_SIZE)
last_label = "Analyzing..."
last_confidence = 0.0

cap = cv2.VideoCapture(0)
print("[INFO] Press 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_crop = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)

        eyes = eye_cascade.detectMultiScale(gray[y:y+h, x:x+w])
        if len(eyes) >= 2:  # at least two eyes detected
            blink_counter += 1
        else:
            if blink_counter >= CONSEC_FRAMES:
                total_blinks += 1
            blink_counter = 0

        try:
            face_resized = cv2.resize(face_crop, (IMG_SIZE, IMG_SIZE))
            face_resized = cv2.cvtColor(face_resized, cv2.COLOR_BGR2RGB)
            face_resized = img_to_array(face_resized)
            face_resized = np.expand_dims(face_resized, axis=0) / 255.0
            prediction = model.predict(face_resized, verbose=0)[0][0]
            prediction_queue.append(prediction)

            if len(prediction_queue) == PREDICTION_HISTORY_SIZE:
                avg_pred = np.mean(prediction_queue)
                blink_rate = total_blinks / ((time.time() - start_time) / 60)
                if avg_pred >= 0.5 and blink_rate >= MIN_BLINK_RATE:
                    last_label = "Real"
                    last_confidence = avg_pred * 100
                else:
                    last_label = "Fake"
                    last_confidence = (1 - avg_pred) * 100

        except Exception as e:
            print(f"[WARN] Prediction error: {e}")

        color = (0, 255, 0) if last_label == "Real" else (0, 0, 255)
        cv2.putText(frame, f"{last_label} ({last_confidence:.2f}%)", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        cv2.putText(frame, f"Blinks: {total_blinks}", (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        blink_rate = total_blinks / ((time.time() - start_time) / 60)
        cv2.putText(frame, f"Blink Rate: {blink_rate:.2f}/min", (10, 110),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    cv2.imshow("Stable Liveness Detection (No Mediapipe)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Press 'q' to quit.


In [18]:
import cv2
import numpy as np
import time
from collections import deque, defaultdict
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from scipy.spatial import distance as dist

# Thresholds
EAR_THRESHOLD = 0.21
CONSEC_FRAMES = 3
MIN_BLINK_RATE = 1
STABLE_PREDICTION_WINDOW = 2.0
IMG_SIZE = 64
MAX_HISTORY = 10
BLUR_THRESHOLD = 150.0
TEXTURE_THRESHOLD = 15.0
BRIGHTNESS_THRESHOLD = 60.0

# EAR calculation
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def calculate_blur(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def calculate_texture(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.std(gray)

def calculate_brightness(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    _, _, v = cv2.split(hsv)
    return np.mean(v)

# Load model
model = load_model("liveness_model.h5")

# Haar cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Track data per face
face_data = defaultdict(lambda: {
    "blink_counter": 0,
    "total_blinks": 0,
    "predictions": deque(maxlen=MAX_HISTORY),
    "last_label": "Analyzing...",
    "last_confidence": 0.0,
    "last_update_time": 0.0,
    "start_time": time.time()
})

def update_prediction(face_id, pred, blink_rate, blur, texture, brightness, current_time):
    data = face_data[face_id]
    data["predictions"].append(pred)
    avg_pred = np.mean(data["predictions"])

    if current_time - data["last_update_time"] > STABLE_PREDICTION_WINDOW:
        is_real_model = avg_pred >= 0.5
        is_real_blink = blink_rate >= MIN_BLINK_RATE
        is_real_texture = texture > TEXTURE_THRESHOLD or (brightness > BRIGHTNESS_THRESHOLD and texture > TEXTURE_THRESHOLD * 0.75)
        is_low_light = brightness < BRIGHTNESS_THRESHOLD

        if is_real_model and (is_real_blink or is_real_texture):
            data["last_label"] = "Real"
            data["last_confidence"] = avg_pred * 100
        elif is_low_light and is_real_model and is_real_texture:
            data["last_label"] = "Real"
            data["last_confidence"] = avg_pred * 100
        else:
            data["last_label"] = "Fake"
            data["last_confidence"] = (1 - avg_pred) * 100

        data["last_update_time"] = current_time

# Start video
cap = cv2.VideoCapture(0)
print("[INFO] Press 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for idx, (x, y, w, h) in enumerate(faces):
        face_id = f"face_{idx}"
        face_crop = frame[y:y+h, x:x+w]

        # Eye detection
        eyes = eye_cascade.detectMultiScale(gray[y:y+h, x:x+w])
        if len(eyes) >= 2:
            face_data[face_id]["blink_counter"] += 1
        else:
            if face_data[face_id]["blink_counter"] >= CONSEC_FRAMES:
                face_data[face_id]["total_blinks"] += 1
            face_data[face_id]["blink_counter"] = 0

        elapsed = time.time() - face_data[face_id]["start_time"]
        blink_rate = face_data[face_id]["total_blinks"] / (elapsed / 60) if elapsed > 0 else 0

        try:
            # Model prediction
            face_input = cv2.resize(face_crop, (IMG_SIZE, IMG_SIZE))
            face_rgb = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
            face_input = img_to_array(face_rgb)
            face_input = np.expand_dims(face_input, axis=0) / 255.0
            prediction = model.predict(face_input, verbose=0)[0][0]

            # Quality checks
            blur_val = calculate_blur(face_crop)
            texture_val = calculate_texture(face_crop)
            brightness_val = calculate_brightness(face_crop)

            update_prediction(face_id, prediction, blink_rate, blur_val, texture_val, brightness_val, time.time())

            # Draw
            label = face_data[face_id]["last_label"]
            confidence = face_data[face_id]["last_confidence"]
            color = (0, 255, 0) if label == "Real" else (0, 0, 255)

            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f"{label} ({confidence:.1f}%)", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            cv2.putText(frame, f"Blinks: {face_data[face_id]['total_blinks']}", (x, y + h + 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
            cv2.putText(frame, f"Rate: {blink_rate:.1f}/min", (x, y + h + 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
            cv2.putText(frame, f"Blur: {blur_val:.1f}", (x, y + h + 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
            cv2.putText(frame, f"Texture: {texture_val:.1f}", (x, y + h + 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
            cv2.putText(frame, f"Brightness: {brightness_val:.1f}", (x, y + h + 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

        except Exception as e:
            print("Error:", e)
            continue

    cv2.imshow("Multi-Face Liveness Detection (No Mediapipe)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Press 'q' to quit.


In [20]:
import cv2
import numpy as np
import time
from collections import deque, defaultdict
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from scipy.spatial import distance as dist

# Parameters
EAR_THRESHOLD = 0.21
CONSEC_FRAMES = 3
MIN_BLINK_RATE = 2
STABLE_PREDICTION_WINDOW = 2.0
IMG_SIZE = 64
MAX_HISTORY = 10
BLUR_THRESHOLD = 150.0
TEXTURE_THRESHOLD = 10.0
BRIGHTNESS_THRESHOLD = 50.0

# Load liveness model
model = load_model("liveness_model.h5")

# Load OpenCV face and eye detectors
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# Data storage for multiple faces
face_data = defaultdict(lambda: {
    "blink_counter": 0,
    "total_blinks": 0,
    "predictions": deque(maxlen=MAX_HISTORY),
    "last_label": "Analyzing...",
    "last_confidence": 0.0,
    "last_update_time": 0.0,
    "start_time": time.time()
})

# Helper functions
def eye_aspect_ratio(eye):
    if len(eye) < 6:
        return 0
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    if C == 0:
        return 0
    return (A + B) / (2.0 * C)

def calculate_blur(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def calculate_texture(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.std(gray)

def calculate_brightness(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    _, _, v = cv2.split(hsv)
    return np.mean(v)

def remap_confidence(label, confidence):
    if label == "Real":
        return 90 + (confidence / 100 * 10)
    else:
        return 60 + (confidence / 100 * 10)

def update_prediction(face_id, pred, blink_rate, blur, texture, brightness, current_time):
    data = face_data[face_id]
    data["predictions"].append(pred)
    avg_pred = np.mean(data["predictions"])

    if current_time - data["last_update_time"] > STABLE_PREDICTION_WINDOW:
        is_real_model = avg_pred >= 0.5
        is_real_blink = blink_rate >= MIN_BLINK_RATE
        is_real_texture = texture > TEXTURE_THRESHOLD or (brightness > BRIGHTNESS_THRESHOLD and texture > TEXTURE_THRESHOLD * 0.75)
        is_low_light = brightness < BRIGHTNESS_THRESHOLD

        if is_real_blink and is_real_texture and not is_low_light:
            data["last_label"] = "Real"
        elif is_real_model and (is_real_blink or is_real_texture):
            data["last_label"] = "Real"
        elif is_low_light and is_real_model and is_real_texture:
            data["last_label"] = "Real"
        else:
            data["last_label"] = "Fake"

        data["last_confidence_raw"] = avg_pred * 100
        data["last_confidence"] = remap_confidence(data["last_label"], avg_pred * 100)
        data["last_update_time"] = current_time

# Start video capture
cap = cv2.VideoCapture(0)
print("[INFO] Press 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for idx, (x, y, w, h) in enumerate(faces):
        face_id = f"face_{idx}"
        face_crop = frame[y:y+h, x:x+w]

        # Eye detection inside face
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        eye_points = []
        for (ex, ey, ew, eh) in eyes[:2]:  # Use max 2 eyes
            eye_center = (ex + ew//2, ey + eh//2)
            eye_points.append(eye_center)
            cv2.rectangle(frame, (x+ex, y+ey), (x+ex+ew, y+ey+eh), (0,255,0), 1)

        ear = eye_aspect_ratio(eye_points)
        if ear < EAR_THRESHOLD:
            face_data[face_id]["blink_counter"] += 1
        else:
            if face_data[face_id]["blink_counter"] >= CONSEC_FRAMES:
                face_data[face_id]["total_blinks"] += 1
            face_data[face_id]["blink_counter"] = 0

        elapsed = time.time() - face_data[face_id]["start_time"]
        blink_rate = face_data[face_id]["total_blinks"] / (elapsed / 60) if elapsed > 0 else 0

        # Prepare face for liveness model
        try:
            face_input = cv2.resize(face_crop, (IMG_SIZE, IMG_SIZE))
            face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
            face_input = img_to_array(face_input)
            face_input = np.expand_dims(face_input, axis=0) / 255.0
            prediction = model.predict(face_input, verbose=0)[0][0]

            blur_val = calculate_blur(face_crop)
            texture_val = calculate_texture(face_crop)
            brightness_val = calculate_brightness(face_crop)

            update_prediction(face_id, prediction, blink_rate, blur_val, texture_val, brightness_val, time.time())

            label = face_data[face_id]["last_label"]
            confidence = face_data[face_id]["last_confidence"]
            color = (0, 255, 0) if label=="Real" else (0,0,255)

            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f"{label} ({confidence:.1f}%)", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            cv2.putText(frame, f"Blinks: {face_data[face_id]['total_blinks']}", (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)
            cv2.putText(frame, f"Rate: {blink_rate:.1f}/min", (x, y+h+40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)
            cv2.putText(frame, f"Blur: {blur_val:.1f}", (x, y+h+60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)
            cv2.putText(frame, f"Texture: {texture_val:.1f}", (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)
            cv2.putText(frame, f"Brightness: {brightness_val:.1f}", (x, y+h+100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)

        except Exception as e:
            print("Error:", e)
            continue

    cv2.imshow("Liveness Detection (OpenCV Only)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Press 'q' to quit.


In [2]:
import cv2
import numpy as np
import time
from collections import deque, defaultdict
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from scipy.spatial import distance as dist
import urllib.request
import os

# ---------------- Parameters ----------------
EAR_THRESHOLD = 0.21
CONSEC_FRAMES = 3
MIN_BLINK_RATE = 2
STABLE_PREDICTION_WINDOW = 2.0
IMG_SIZE = 64
MAX_HISTORY = 10
TEXTURE_THRESHOLD = 10.0
BRIGHTNESS_THRESHOLD = 50.0

# ---------------- Eye Aspect Ratio ----------------
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

# ---------------- Image Metrics ----------------
def calculate_blur(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def calculate_texture(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.std(gray)

def calculate_brightness(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    _, _, v = cv2.split(hsv)
    return np.mean(v)

# ---------------- Load Liveness Model ----------------
model = load_model("liveness_model.h5")

# ---------------- Download DNN Face Detector if not exist ----------------
model_dir = r"C:\models\face_detector"
os.makedirs(model_dir, exist_ok=True)

proto_path = os.path.join(model_dir, "deploy.prototxt")
model_path = os.path.join(model_dir, "res10_300x300_ssd_iter_140000.caffemodel")

if not os.path.isfile(proto_path):
    print("[INFO] Downloading deploy.prototxt...")
    urllib.request.urlretrieve(
        "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt",
        proto_path
    )

if not os.path.isfile(model_path):
    print("[INFO] Downloading res10_300x300_ssd_iter_140000.caffemodel...")
    urllib.request.urlretrieve(
        "https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel",
        model_path
    )

face_net = cv2.dnn.readNetFromCaffe(proto_path, model_path)

# ---------------- Eye Cascade for Blink Detection ----------------
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# ---------------- Face Data ----------------
face_data = defaultdict(lambda: {
    "blink_counter": 0,
    "total_blinks": 0,
    "predictions": deque(maxlen=MAX_HISTORY),
    "last_label": "Analyzing...",
    "last_confidence": 0.0,
    "last_update_time": 0.0,
    "start_time": time.time()
})

def remap_confidence(label, confidence):
    if label == "Real":
        return 90 + (confidence / 100 * 10)
    else:
        return 60 + (confidence / 100 * 10)

def update_prediction(face_id, pred, blink_rate, blur, texture, brightness, current_time):
    data = face_data[face_id]
    data["predictions"].append(pred)
    avg_pred = np.mean(data["predictions"])

    if current_time - data["last_update_time"] > STABLE_PREDICTION_WINDOW:
        is_real_model = avg_pred >= 0.5
        is_real_blink = blink_rate >= MIN_BLINK_RATE
        is_real_texture = texture > TEXTURE_THRESHOLD or (brightness > BRIGHTNESS_THRESHOLD and texture > TEXTURE_THRESHOLD * 0.75)
        is_low_light = brightness < BRIGHTNESS_THRESHOLD

        if is_real_blink and is_real_texture and not is_low_light:
            data["last_label"] = "Real"
        elif is_real_model and (is_real_blink or is_real_texture):
            data["last_label"] = "Real"
        elif is_low_light and is_real_model and is_real_texture:
            data["last_label"] = "Real"
        else:
            data["last_label"] = "Fake"

        data["last_confidence_raw"] = avg_pred * 100
        data["last_confidence"] = remap_confidence(data["last_label"], avg_pred * 100)
        data["last_update_time"] = current_time

# ---------------- Webcam Loop ----------------
cap = cv2.VideoCapture(0)
print("[INFO] Press 'q' to quit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    # ---------------- DNN Face Detection ----------------
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
                                 (300, 300), (104.0, 177.0, 123.0))
    face_net.setInput(blob)
    detections = face_net.forward()

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence < 0.5:
            continue

        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (x_min, y_min, x_max, y_max) = box.astype("int")
        face_crop = frame[y_min:y_max, x_min:x_max]

        if face_crop.size == 0:
            continue

        # ---------------- Liveness Prediction ----------------
        face_input = cv2.resize(face_crop, (IMG_SIZE, IMG_SIZE))
        face_rgb = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
        face_input = img_to_array(face_rgb)
        face_input = np.expand_dims(face_input, axis=0) / 255.0
        pred = model.predict(face_input, verbose=0)[0][0]

        blur_val = calculate_blur(face_crop)
        texture_val = calculate_texture(face_crop)
        brightness_val = calculate_brightness(face_crop)

        face_id = f"face_{i}"
        elapsed = time.time() - face_data[face_id]["start_time"]
        blink_rate = face_data[face_id]["total_blinks"] / (elapsed / 60) if elapsed > 0 else 0

        update_prediction(face_id, pred, blink_rate, blur_val, texture_val, brightness_val, time.time())

        label = face_data[face_id]["last_label"]
        conf = face_data[face_id]["last_confidence"]
        color = (0, 255, 0) if label == "Real" else (0, 0, 255)

        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
        cv2.putText(frame, f"{label} ({conf:.1f}%)", (x_min, y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Blur: {blur_val:.1f}", (x_min, y_max + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        cv2.putText(frame, f"Texture: {texture_val:.1f}", (x_min, y_max + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        cv2.putText(frame, f"Brightness: {brightness_val:.1f}", (x_min, y_max + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    cv2.imshow("Liveness Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Press 'q' to quit.


In [1]:
import cv2
import numpy as np
import time
from collections import deque, defaultdict
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from scipy.spatial import distance as dist

# ---------------- Parameters ----------------
EAR_THRESHOLD = 0.21
CONSEC_FRAMES = 3
MIN_BLINK_RATE = 2
STABLE_PREDICTION_WINDOW = 2.0
IMG_SIZE = 64
MAX_HISTORY = 10
TEXTURE_THRESHOLD = 10.0
BRIGHTNESS_THRESHOLD = 40.0  # more lenient for low light

# ---------------- Functions ----------------
def calculate_blur(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def calculate_texture(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return np.std(gray)

def calculate_brightness(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    _, _, v = cv2.split(hsv)
    return np.mean(v)

def remap_confidence(label, confidence):
    if label == "Real":
        return 90 + (confidence / 100 * 10)  # 90–100%
    else:
        return 60 + (confidence / 100 * 10)  # 60–70%

def update_prediction(face_id, pred, blink_rate, texture, brightness, current_time):
    data = face_data[face_id]
    data["predictions"].append(pred)
    avg_pred = np.mean(data["predictions"])

    if current_time - data["last_update_time"] > STABLE_PREDICTION_WINDOW:
        is_real_model = avg_pred >= 0.5
        is_real_blink = blink_rate >= MIN_BLINK_RATE
        is_real_texture = texture > TEXTURE_THRESHOLD or (brightness > BRIGHTNESS_THRESHOLD and texture > TEXTURE_THRESHOLD * 0.75)
        is_low_light = brightness < BRIGHTNESS_THRESHOLD

        if is_real_blink and is_real_texture and not is_low_light:
            data["last_label"] = "Real"
        elif is_real_model and (is_real_blink or is_real_texture):
            data["last_label"] = "Real"
        elif is_low_light and is_real_model and is_real_texture:
            data["last_label"] = "Real"
        else:
            data["last_label"] = "Fake"

        data["last_confidence_raw"] = avg_pred * 100
        data["last_confidence"] = remap_confidence(data["last_label"], avg_pred * 100)
        data["last_update_time"] = current_time

# ---------------- Load Models ----------------
model = load_model("liveness_model.h5")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# ---------------- Data ----------------
face_data = defaultdict(lambda: {
    "blink_counter": 0,
    "total_blinks": 0,
    "predictions": deque(maxlen=MAX_HISTORY),
    "last_label": "Analyzing...",
    "last_confidence": 0.0,
    "last_update_time": 0.0,
    "start_time": time.time()
})

# ---------------- Capture ----------------
cap = cv2.VideoCapture(0)
print("[INFO] Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for i, (x, y, w, h) in enumerate(faces):
        face_crop = frame[y:y+h, x:x+w]

        # Prepare for model prediction
        face_input = cv2.resize(face_crop, (IMG_SIZE, IMG_SIZE))
        face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
        face_input = img_to_array(face_input)
        face_input = np.expand_dims(face_input, axis=0) / 255.0
        pred = model.predict(face_input, verbose=0)[0][0]

        # Blink detection
        gray_face = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
        eyes = eye_cascade.detectMultiScale(gray_face, scaleFactor=1.1, minNeighbors=5)
        ear = len(eyes) / 2.0  # simple proxy for blink

        face_id = f"face_{i}"
        elapsed = time.time() - face_data[face_id]["start_time"]
        blink_rate = face_data[face_id]["total_blinks"] / (elapsed / 60) if elapsed > 0 else 0

        # Feature calculations
        blur_val = calculate_blur(face_crop)
        texture_val = calculate_texture(face_crop)
        brightness_val = calculate_brightness(face_crop)

        update_prediction(face_id, pred, blink_rate, texture_val, brightness_val, time.time())

        label = face_data[face_id]["last_label"]
        conf = face_data[face_id]["last_confidence"]
        color = (0, 255, 0) if label == "Real" else (0, 0, 255)

        # Display
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, f"{label} ({conf:.1f}%)", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, f"Blinks: {face_data[face_id]['total_blinks']}", (x, y+h+20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0),1)
        cv2.putText(frame, f"Brightness: {brightness_val:.1f}", (x, y+h+40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255),1)
        cv2.putText(frame, f"Texture: {texture_val:.1f}", (x, y+h+60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),1)

    cv2.imshow("Offline Liveness Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Press 'q' to quit.
